In [11]:
from IPython.html.services.config import ConfigManager
from IPython.utils.path import locate_profile
cm = ConfigManager(profile_dir=locate_profile(get_ipython().profile))
cm.set('notebook', {'load_extensions': {'usability/hide_input': True}})
cm.update('notebook', {'load_extensions': {'livereveal/main': True}})
cm.update('notebook', {'load_extensions': {'usability/comment-uncomment': True}})
cm.update('notebook', {'load_extensions': {'usability/codefolding/main': True}})
cm.set('livereveal', {'transition': 'none', 'theme': 'moon'})
cm.update('livereveal', {'start_slideshow_at': 'selected'})
cm.get('notebook')
cm.get('livereveal')

{u'start_slideshow_at': u'selected', u'theme': u'moon', u'transition': u'none'}

In [4]:
import numpy as np
import toyplot

<h1 style="text-align: left;font-size: 300%">
    <b><font color="#f08">Multilevel</font></b>
</h1>
<h1 style="text-align: left;font-size: 300%">
    <b>Monte Carlo</b>
</h1>
<h1 style="text-align: left;font-size: 300%">
    <b>Tutorial</b>
</h1>

<hr style="height:2px;border:none;color:#333;background-color:#333;width=60%" />

<br>

<h1 style="text-align: left; font-size: 200%;">
    <b>Chris Ketelsen</b>
<h1 style="text-align: left; font-size: 200%;">
    <b>Univ. of Colorado at Boulder</b>
</h1>

**About Me**: 

- PhD in Applied Math at CU Boulder in 2009 (Numerical PDEs/Linear Algebra, Multigrid Methods)
- Postdoc at Lawrence Livermore Nat'l Lab 2010-2013 (Multigrid Methods, Multilevel Monte Carlo)
- Instructor in Applied Math at CU Boulder 2013-2016 
- Instructor in Computer Science at CU Boulder 2016-
 

**About Me**: 

- Numerical linear algebraist by training
- Know just enough probability/statistics to be a danger to myself (and others) 

<br>
<br>
<br>

**Outline**:

- Part 1: Monte Carlo Preliminaries
- Part 2: Multilevel Monte Carlo - The General Idea 
- Part 3: Multilevel Monte Carlo - Implementation Details
- Part 4: Multilevel Markov Chain Monte Carlo 

<h1 style="text-align: left;font-size: 300%">
    <b><font color="#f08">Multilevel</font></b>
</h1>
<h1 style="text-align: left;font-size: 300%">
    <b>Monte Carlo</b>
</h1>
<h1 style="text-align: left;font-size: 300%">
    <b>Tutorial</b>
</h1>

<hr style="height:2px;border:none;color:#333;background-color:#333;width=60%" />

<h1 style="text-align: left; font-size: 300%;">
    <b>Part 1: Monte Carlo Preliminaries</b>
</h1>

*https://en.wikipedia.org/wiki/Monte_Carlo_method*:

Monte Carlo Methods vary, but tend to follow a particular pattern: 

1. Define a domain of possible inputs
2. Generate inputs randomly  from a probability distribution
3. Perform a **deterministic** computation on the inputs
4. Aggregate the results

<br>

*https://en.wikipedia.org/wiki/Monte_Carlo_method*:

Monte Carlo Methods vary, but tend to follow a particular pattern: 

1. Define a domain of possible inputs
2. Generate inputs randomly  from a probability distribution
3. Perform a **deterministic** computation on the inputs
4. Aggregate the results

Step 1: Usually fixed by the application

*https://en.wikipedia.org/wiki/Monte_Carlo_method*:

Monte Carlo Methods vary, but tend to follow a particular pattern: 

1. Define a domain of possible inputs
2. Generate inputs randomly  from a probability distribution
3. Perform a **deterministic** computation on the inputs
4. Aggregate the results

Step 2: Sometimes easy, sometimes hard (Markov Chains / Part 4)

*https://en.wikipedia.org/wiki/Monte_Carlo_method*:

Monte Carlo Methods vary, but tend to follow a particular pattern: 

1. Define a domain of possible inputs
2. Generate inputs randomly  from a probability distribution
3. Perform a **deterministic** computation on the inputs
4. Aggregate the results

Step 3: Usually the expensive part (ODE/PDE solves, etc)

*https://en.wikipedia.org/wiki/Monte_Carlo_method*:

Monte Carlo Methods vary, but tend to follow a particular pattern: 

1. Define a domain of possible inputs
2. Generate inputs randomly  from a probability distribution
3. Perform a **deterministic** computation on the inputs
4. Aggregate the results

Step 4: Clever tricks lead to major cost savings (Variance Reduction)

**Classic Example**: Estimating $\pi$

<br>

1. Inscribe a circle of radius 1 in a square
2. Generate points uniformly random in the square
3. Count the number of points inside the circle
4. Ratio of points in circle to points in square $\approx ~ \frac{\pi}{4}$

<br>
<br>
<br>

**Classic Example**: Estimating $\pi$



In [5]:
np.random.seed(1235)
x = np.random.uniform(-1,1,50)
y = np.random.uniform(-1,1,50)
ind = x*x + y*y < 1 
t = np.linspace(0,2*np.pi,200)
canvas = toyplot.Canvas(width=350, height=350)
axes = canvas.axes()
axes.show = False
axes.scatterplot(x[ind], y[ind], marker="o", size=8);
axes.scatterplot(x[~ind], y[~ind], marker="o", size=8);
axes.plot(np.cos(t), np.sin(t));
axes.plot([-1,1,1,-1,-1],[1,1,-1,-1,1]);

<path d="M 300.0 175.0 L 299.93769864077171 171.05393127939868 L 299.75085666643668 167.11179608591578 L 299.43966032513828 163.17752402564437 L 299.00441982415725 159.25503686653533 L 298.44556902069047 155.34824462909364 L 297.76366498937256 151.46104168878529 L 296.95938746697169 147.59730289403859 L 296.03353817481337 143.76087970371145 L 294.98704001960715 139.95559634787321 L 293.82093617347283 136.18524601572892 L 292.53638903408444 132.45358707448591 L 291.13467906596628 128.76433932293165 L 289.61720352409861 125.12118028345738 L 287.98547506110299 121.5277415362238 L 286.24112021939749 117.987605099123 L 284.38587780982419 114.50429985714514 L 282.42159717836404 111.08129804470917 L 280.3502363626688 107.72201178446393 L 278.17386014024709 104.42978968601025 L 275.89463797024814 101.20791350793397 L 273.51484183089906 98.059594886477598 L 271.03684395474636 94.987972134111601 L 268.46311446396362 91.99610711119611 L 265.79621890807994 89.086982173851993 L 263.03881570658399 86.263497201083396 L 260.19365349895423 83.528466704115402 L 257.26356840475472 80.884617020827875 L 254.25148119652982 78.334583598082602 L 251.16039438831459 75.880908364652683 L 247.99338924266328 73.526037197372403 L 244.75362269918003 71.272317483034271 L 241.4443242276127 69.121995778462633 L 238.06879260864721 67.077215571097099 L 234.63039264561121 65.140015142317452 L 231.13255181036502 63.312325535640738 L 227.57875682672301 61.595968631814834 L 223.97255019481165 59.992655332728198 L 220.31752665982808 58.503983855945641 L 216.61732962872014 57.131438141570356 L 212.87564753835909 55.876386373020097 L 209.0962101788254 54.740079613192151 L 205.28278497547291 53.723650557376729 L 201.43917323347739 52.828112404161587 L 197.56920634861279 52.054357845453772 L 193.67674198803263 51.403158176624672 L 189.76566024486351 50.875162527666284 L 185.83985977044384 50.470897216124143 L 181.90325388806349 50.190765222452654 L 177.95976669207801 50.035045788315259 L 174.01332913628585 50.003894138230457 L 170.06787511546844 50.097341324840542 L 166.12733754399801 50.31529419795767 L 162.19564443542333 50.657535497417875 L 158.27671498694011 51.123724069650997 L 154.37445567265007 51.713395207749663 L 150.49275634950274 52.425961114699504 L 146.63548637980125 53.260711489307788 L 142.8064907741379 54.216814234247281 L 139.00958635860422 55.293316285508801 L 135.24855797009525 56.489144562436323 L 131.5271546835022 57.80310703739687 L 127.84908607455301 59.233893924019853 L 124.21801852202637 60.780078982820406 L 120.63757155302588 62.440120942906027 L 117.11131423495652 64.212365038348906 L 113.64276161780083 66.09504465769254 L 110.2353712302403 68.086283104948535 L 106.8925396331162 70.184095470327847 L 103.61759903366317 72.386390608842177 L 100.41381396389303 74.690973224802747 L 97.284378026438119 77.095546060138702 L 94.232410711098765 79.597712184353952 L 91.260954285268099 82.194977383839614 L 88.372970761333534 84.884752648160429 L 85.571338944078605 87.664356750836674 L 82.858851561027635 90.531018922048901 L 80.238212478594335 93.481881610601647 L 77.71203400680875 96.51400333239269 L 75.282834295310096 99.624361602548191 L 72.953034823200127 102.80985594830142 L 70.724957985260318 106.06731099961155 L 68.600824776938197 109.39347965444145 L 66.58275258041067 112.78504631553963 L 64.672753053931643 116.23863019549964 L 62.872730126567063 119.75078868680262 L 61.184478100317229 123.31802079348324 L 59.609679861517421 126.93677062099863 L 58.149905203300222 130.60343092082144 L 56.80660926079161 134.31434668622356 L 55.581131060600654 138.06581879566625 L 54.474692186048536 141.85410770016517 L 53.488395559467875 145.67543715095388 L 52.623224342785697 149.52599796373096 L 51.880040957486187 153.40195181573759 L 51.259586224930359 157.29943507188105 L 50.762478627889024 161.21456263609062 L 50.389213694025855 165.14343182406543 L 50.140163501944549 169.08212625355523 L 50.015576310292836 173.02671974829542 L 50.015576310292836 176.9732802517

In [6]:
np.random.seed(1235)
N = 50
x = np.random.uniform(-1,1,N)
y = np.random.uniform(-1,1,N)
ind = x*x + y*y < 1
p = 4.0 * np.sum(ind) / N
print p
print abs(p-np.pi)/np.pi

3.2
0.0185916357881


With $n = 50$ points, $40$ inside circle, $\pi \approx 4(0.8) = 3.2$ ($2\%$ relative error)

With $n = 5000$ points, $\pi \approx  3.1512$ ($0.3\%$ relative error)  

**Two Take-Aways From This**: 

1. It works!  
2. It's really really slow...

Going from $n=50$ to $n=5000$ requires $100$ times the work to go from one correct digit to two correct digits

**Classic Example**: Estimating $\pi$
 
$~~~$1.$~$ Domain ${\cal D} = [-1,1] \times [-1,1]$

$~~~$2.$~$ Sample $x_i$ and $y_i$ from $U[-1,1]$ for $i=1,\ldots,N$

$~~~$3.$~$ For each $i$, compute   

$$
H_i = H\left(x_i, y_i\right) = \left\{ 
\begin{array}{ll}
1 & \textrm{if} ~~ x_i^2 + y_i^2 < 1 \\
0 & \textrm{otherwise}
\end{array}
\right.
$$

$~~~$4.$~$ Estimate

$$
\pi \approx 4 \cdot \frac{1}{N}\sum_{i=1}^n H_i 
$$


<br>
<br>
<br>

**Many Flavors of Monte Carlo**: 
    
- **Simulation and Optimization**:  Think of state space as probability distribution and look for a global maximum (or pretty good local maximum)

- **Inverse Problems**: Given some real-world data and a probability distribution over some model space, try to find the model that data most likely came from

- **Integration**: Deterministic integration and Monte Carlo estimation are naturally linked.  We'll use this idea throughout Part 1 of the tutorial. 

In [7]:
np.log(2)

0.69314718055994529

**Monte Carlo Integration**:

Let's look at the simple example of approximating

$$
I = \int_0^1 f(x) ~dx = \int_0^1 \frac{1}{1+x} ~dx = \ln 2 = 0.69314718
$$

Could use numerical quadrature (e.g. Composite Trapezoid Rule)

In [8]:
x = np.linspace(0,1,2**5+2)
y = 1./(1 + x) 
canvas = toyplot.Canvas(width=830, height=300)
axes1 = canvas.cartesian(bounds=(30, 370, 30, 270))
axes1.plot(x,y);
axes1.plot([0,1],[0,0], color="white")
axes2 = canvas.cartesian(bounds=(460, 800, 30, 270))
axes2.plot(x,y);
axes2.plot([0,1],[0,0], color="steelblue")
x = np.linspace(0,1,2**3)
z = np.zeros(x.size)
for ii in xrange(x.size): 
    axes2.plot([x[ii],x[ii]], [0,1/(1 + x[ii])], marker="o", style={"stroke-width":3}, color="steelblue");
axes2.plot(x, 1/(1 + x), color="steelblue");

<path d="M 30.0 30.0 L 40.303030303030305 37.058823529411768 L 50.606060606060609 43.714285714285694 L 60.909090909090914 49.999999999999986 L 71.212121212121218 55.945945945945923 L 81.515151515151516 61.578947368421069 L 91.818181818181827 66.92307692307692 L 102.12121212121212 72.000000000000014 L 112.42424242424244 76.829268292682912 L 122.72727272727272 81.428571428571431 L 133.03030303030303 85.813953488372078 L 143.33333333333334 90.000000000000028 L 153.63636363636365 94.000000000000014 L 163.93939393939394 97.826086956521749 L 174.24242424242425 101.48936170212765 L 184.54545454545456 105.0 L 194.84848484848487 108.36734693877551 L 205.15151515151513 111.59999999999999 L 215.45454545454544 114.70588235294116 L 225.75757575757578 117.69230769230768 L 236.06060606060606 120.56603773584904 L 246.36363636363635 123.33333333333331 L 256.66666666666669 126.0 L 266.96969696969694 128.57142857142856 L 277.27272727272731 131.05263157894737 L 287.57575757575756 133.44827586206895 L 297.87878787878788 135.76271186440675 L 308.18181818181819 138.0 L 318.4848484848485 140.16393442622953 L 328.78787878787875 142.25806451612902 L 339.09090909090912 144.28571428571428 L 349.39393939393943 146.25 L 359.69696969696975 148.15384615384616 L 370.0 150.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0.0 0.5 1.0 0.0 0.5 1.0 <path d="M 460.0 30.0 L 470.30303030303031 37.058823529411768 L 480.60606060606062 43.714285714285694 L 490.90909090909093 49.999999999999986 L 501.21212121212125 55.945945945945923 L 511.5151515151515 61.578947368421069 L 521.81818181818176 66.92307692307692 L 532.12121212121212 72.000000000000014 L 542.42424242424249 76.829268292682912 L 552.72727272727275 81.428571428571431 L 563.030303030303 85.813953488372078 L 573.33333333333326 90.000000000000028 L 583.63636363636374 94.000000000000014 L 593.93939393939399 97.826086956521749 L 604.24242424242425 101.48936170212765 L 614.5454545454545 105.0 L 624.84848484848487 108.36734693877551 L 635.15151515151513 111.59999999999999 L 645.4545454545455 114.70588235294116 L 655.75757575757575 117.69230769230768 L 666.06060606060612 120.56603773584904 L 676.36363636363637 123.33333333333331 L 686.66666666666663 126.0 L 696.969696969697 128.57142857142856 L 707.27272727272725 131.05263157894737 L 717.57575757575751 133.44827586206895 L 727.87878787878788 135.76271186440675 L 738.18181818181824 138.0 L 748.4848484848485 140.16393442622953 L 758.78787878787875 142.25806451612902 L 769.09090909090912 144.28571428571428 L 779.39393939393938 146.25 L 789.69696969696963 148.15384615384616 L 800.0 150.0" style="stroke:rgb(40%,76.1%,64.7%);stroke-opacity:1.0;stroke-width:2.0"> 0.0 0.5 1.0 0.0 0.5 1.0 
 
 
 Save as .csv

**Monte Carlo Integration**:

Let's look at the simple example of approximating

$$
I = \int_0^1 f(x) ~dx = \int_0^1 \frac{1}{1+x} ~dx = \ln 2 = 0.69314718
$$

Could use numerical quadrature (e.g. Composite Trapezoid Rule)

$$
I \approx T_N = \frac{1}{N}\left( ~~ \frac{1}{2}f_1 + f_2 + \cdots + f_{N-1} + \frac{1}{2}f_N ~~\right) ~~ \textrm{where} ~~ f_i = f(x_i)
$$

Which does pretty well: $T_{32} = 0.69321220$ ($\left|I-T_{32}\right| = 6.5 \times 10^{-5}$) at cost of 32 function evaluations

<br>

**Monte Carlo Integration**:

We can approximate the same integral via Monte Carlo

Instead of averaging function evals at equispaced points, we use uniformly random points sampled throughout the domain

$$
I \approx \hat{I}^{MC}_N = \frac{1}{N}\sum_{i=1}^N \frac{1}{1 + X_i} ~~ \textrm{where} ~~ X_i ~~ \textrm{iid from} ~~ \mathcal{U}(0,1)
$$

$$
\begin{array}{|c|c|c|}
\hline
N & \hat{I}^{MC}_N & \left| ~ I - \hat{I}^{MC}_N ~ \right| \\
\hline
32 & 0.741660 & 0.048513 \\
\hline
64 & 0.664353 & 0.028793 \\
\hline
5 \times 10^6& 0.693068 & 7.85 \times 10^{-5} \\
\hline
\end{array}
$$


**Monte Carlo Integration**

Same Two Take-Aways From This:

1. It works!  
2. It's really really slow...

Needed 5 **Million** function evaluations to get the same accuracy as Trapezoid Rule with 32 function evaluations. 

Let's deal with the **why** it works and then come back to the slowness... 

In [9]:
def f(x):
    return 1./(1+x)

np.random.seed(1235)

N = 32 
x = np.linspace(0,1,N)
I32 = (1.0/(2*(N-1))) * (2*np.sum(f(x))-f(0)-f(1))
print I32

N = 5000000
x = np.random.uniform(0,1,N)
M32 = np.sum(f(x)) / N
print M32
print np.abs(M32 - np.log(2))

0.693212208525
0.693068613106
7.85674540134e-05


**Monte Carlo Integration**

Let $X$ be a random variable with some probability distribution $P$ 

The two most commonly discussed properties of $X$ are the 

- mean (central tendency, or average, or expected value)
- variance (tells you about the spread of the distribution)

We'll focus on the mean for now 

**Monte Carlo Integration**

If you want to know the long-run average of a random variable you can either estimate it or (if you're lucky) compute it analytically.  

To estimate the mean of a random variable you can draw a bunch of samples of $X$ and take the average 

$$
\hat{X}_N = \frac{1}{N}\sum_{i=1}^N X_i 
$$


**Monte Carlo Integration**

To compute it analytically you can find the *expected value* of $X$

$$
\Bbb{E}[X] = \int X ~ dP = \int_\Omega x ~ \varphi(x) ~ dx 
$$

where here $\Omega$ is the domain of $X$ and $\varphi(x)$ is its *probability density function*

The **Law of Large Numbers** tells us that as you use more and more samples in the estimation, the estimate $\hat{X}_N$ approaches the actual value of $\Bbb{E}[X]$

**Monte Carlo Integration**

Small technical aside... 

Expectation is Linear.  This follows from the fact that expectation is defined as an integral, and integration is linear.  

Let $a$, $b$,$c$ be constants and $X$ and $Y$ be random variables 

$$
\Bbb{E}[aX + bY + c] = a\Bbb{E}[X] + b\Bbb{E}[Y] + c 
$$

We will use this fact a lot

**Monte Carlo Integration**

We can also compute expectations of functions of random variables.

If $f$ is a function and $X$ is a random variable, then $f(X)$ is a random variable as well 

We can estimate its expectation, or compute it analytically 

$$
\hat{f}_N = \frac{1}{N}\sum_{i=1}^N f(X_i) ~~~\textrm{or}~~~ \Bbb{E}[f(X)] = \int_\Omega f(x) \varphi(x) ~ dx
$$

**Monte Carlo Integration**

The notion of the function $f$ here is completely arbitrary.  

It can be as simple as a function evaluation (as in our integration example) or as complicated as solving a partial differential equation 

**Monte Carlo Integration**

In the previous example we had $~f(x) = \dfrac{1}{1+x}$ 

We approximated $I$ by averaging values of $f$ evaluated at points on $[0,1]$ drawn from a uniform distribution

The pdf for ${\cal U}(0,1)$ is $\varphi(x) = 1$, which gives 

$$
\Bbb{E}[f(X)] = \int_0^1 \frac{1}{1+x} \cdot 1 ~ dx \approx \frac{1}{N} \sum_{i=1}^N \frac{1}{1+X_i} ~~ \textrm{where} ~~ X_i ~~ \textrm{iid from} ~~ {\cal U}(0,1)
$$

**Monte Carlo Integration**

OK, so in this particular case, estimating $\Bbb{E}[f(X)]$ is equivalent to estimating the value of the deterministic integral $I$. 

Now we'll look at why the Monte Carlo estimate is so much worse than the Trapezoid Rule

And then I'll try to convince you that in many cases Monte Carlo is actually pretty good 

<br>

**Monte Carlo Accuracy** 

Given a fixed number of samples $N$ in the Monte Carlo estimate, how close do we expect $\hat{f}_N$ to be to $\Bbb{E}[f(X)]$? 

To answer this question we need to introduce a few new definitions, mostly related to variances



**Monte Carlo Accuracy** 

The variance of a random variable is a measure of the spread of samples of the random variable about the mean

**def**: The *variance* of a random variable $X$ is the expected value of of the squared deviation from the mean of $X$

$$
\sigma^2 = \Bbb{V}[X] = \Bbb{E}[(X - \Bbb{E}[X])^2] = \Bbb{E}[X^2] - (\Bbb{E}[X])^2
$$

**def**: The standard deviation of a random variable $X$ is the square root of the variance. 

The standard deviation is a popular because it still gives an indication of the spread of the random variable, but has the same units as the variable itself. 

**Monte Carlo Accuracy**  

**Properties**: Let $a$ be a constant and $X$ be a random variable

$$
\Bbb{V}[X] \geq 0
$$

$$
\Bbb{V}[X + a] = \Bbb{V}[X]
$$

$$
\Bbb{V}[aX] = a^2\Bbb{V}[X]
$$

**Monte Carlo Accuracy** 

**def**: The *covariance* of two random variables is a measure of how much the two random variables change together

$$
\textrm{cov}(X, Y) = \Bbb{E}[(X - \Bbb{E}[X])(Y - \Bbb{E}[Y])] = \Bbb{E}[XY] - \Bbb{E}[X]\Bbb{E}[Y]
$$

**fact**: If random variables $X$ and $Y$ are independent then 

$$
\Bbb{E}[XY] = \Bbb{E}[X]\Bbb{E}[Y]
$$

**fact**: If random variables $X$ and $Y$ are independent then $\textrm{cov}(X,Y) = 0$


**Monte Carlo Accuracy**  

**More Properties**: Let $a$ and $b$ be constants and $X$ and $Y$ be a random variables

$$
\Bbb{V}[aX + bY] = a^2\Bbb{V}[X] + b^2\Bbb{V}[Y] + 2ab ~ \textrm{cov}(X, Y)
$$

We can approximate the variance of a random variable using a finite number of samples 

$$
s^2 = \frac{1}{N-1} \sum_{i=1}^N (X_i - \hat{X}_N)^2
$$

**Monte Carlo Accuracy** 

**fact**: If $f(X)$ is a random variable, then its Monte Carlo estimate is also a random variable 

$$
\hat{f}_N = \frac{1}{N}\sum_{i=1}^N f(X_i)
$$

**fact**: The Monte Carlo estimate is an unbiased estimate of $\Bbb{E}[f(X)]$

$$
\Bbb{E}[\hat{f}_N] = \Bbb{E}\left[ \frac{1}{N}\sum_{i=1}^N f(X_i)\right] = \frac{1}{N}\sum_{i=1}^N \Bbb{E}[f(X_i)] = \Bbb{E}[f(X)]
$$

**Monte Carlo Accuracy** 

The variance of the MC estimate gives us a concrete measure of accuracy as a function of the number of samples, $N$

$$
\Bbb{V}[\hat{f}_N] = \Bbb{E}[(\hat{f}_N - \Bbb{E}[f(X)])^2] = \frac{\Bbb{V}[f(X)]}{N} = \frac{\sigma^2}{N}
$$

**Monte Carlo Accuracy** 

To see this, note that 

$$
\Bbb{V}[\hat{f}_N] = \Bbb{V}\left[ \frac{1}{N} \sum_{i=1}^N f(X_i) \right]
= \frac{1}{N^2}\Bbb{V}\left[\sum_{i=1}^N f(X_i) \right]
$$

$$
= \frac{1}{N^2}\sum_{i=1}^N \Bbb{V}[f(X_i)] = \frac{1}{N^2} ~ N \sigma^2 = \frac{\sigma^2}{N}
$$

(Step 3 follows from the fact that each $f(X_i)$ is independent.)

**Monte Carlo Accuracy** 

Taking the square root gives us the *root mean square error* (RMSE).  This statistic is popular because it gives a measure of the accuracy in the proper units of the estimated quantity 

$$
\textrm{RMSE} = \sqrt{\Bbb{E}[(\hat{f}_N - \Bbb{E}[f(X)])^2]} = \frac{\sigma}{\sqrt{N}} = {\cal O}(N^{-1/2})
$$

This says that to get an improvement in accuracy in RMSE by a factor of 10 you need to increase the number of samples in your MC estimator by a factor of 100! 

Compare this with the Trapezoid Rule which has error that decays like ${\cal O}(N^{-2})$ and you start to question why we thought Monte Carlo was a good idea in the first place ... 

**Monte Carlo Accuracy** 

... until you get to this problem 

$$
I = \int_0^1 \int_0^1 \frac{1}{1 + x + y} ~ dx ~ dy 
$$

With the 2D Composite Trap Rule you need a mesh of equispaced points on the unit square (32 points in each direction is now 1024 points)

**Monte Carlo Accuracy** 

... or this problem 

$$
I = \int_0^1 \int_0^1 \int_0^1 \frac{1}{1 + x + y + z} ~ dx ~ dy ~ dz 
$$

32 points turns into 33K points ...

In general, the Composite Trap Rule in $d$ dimensions has error that decays like ${\cal O}(N^{-2/d})$

This is a clear case of the **Curse of Dimensionality**

**Monte Carlo Accuracy** 

For Monte Carlo, dimensionality is not a problem 

The estimator for the given problem would look something like 

$$
\hat{I}^{MC}_N = \frac{1}{N} \sum_{i=1}^M \frac{1}{1+x_i+y_i+z_i}
$$

where each $x_i$, $y_i$, $z_i$ is an independent draw from ${\cal U}(0,1)$

RMSE still decays like ${\cal O}(N^{-1/2})$

**Two Take-Aways From This**: 

1. Yes, Monte Carlo is slow 
2. For high-dimensional problems, it's quite often the best thing to do 


**Improvements**

If accuracy of Monte Carlo estimation goes like $\frac{\sigma}{\sqrt{N}}$

and we can't do anything about the denominator 

we should consider attacking the numerator 

... **VARIANCE REDUCTION**! 

**Variance Reduction**

The goal of variance reduction is to design new estimators that have the same expectation as vanilla Monte Carlo, but reduce the variance of the simulated random variable 

Here we'll talk about the Method of Control Variates, which shares some similarities with Multilevel Monte Carlo

**Control Variates**

Suppose that we wish to estimate the expectation of $Y = f(X)$ via Monte Carlo for which we know the RMSE will decay like $\sigma / \sqrt{N}$. 

The idea of Control Variates is to define a new random variable, $Y^*$ that has the same expectation as $Y$ but has a smaller variance. 

Suppose you have another random variable $Z$ with *known expectation* $\Bbb{E}[Z]$.  Define 

$$
Y^* = Y + c~(Z - \Bbb{E}[Z])
$$

**Control Variates**

Note that $Y$ and $Y^*$ have the same expectation

$$
\Bbb{E}[Y^*] = \Bbb{E}[Y] + c~\Bbb{E}[(Z - \Bbb{E}[Z])] = \Bbb{E}[Y] + 0 = \Bbb{E}[Y]
$$

The variance of $Y^*$ is given by 

$$
\Bbb{V}[Y^*] = \Bbb{V}[Y] + c^2 \Bbb{V}[Z] + 2c ~ \textrm{cov}(Y, Z)
$$

We can minimize $\Bbb{V}[Y^*]$ by making an intelligent choice for $c$ 

**Control Variates** 

Differentiating w.r.t. $c$ and setting the quantity equal to 0 gives

$$
c^* = - \frac{\textrm{cov}(Y,Z)}{\Bbb{V}[Z]} 
$$

which gives 

$$
\Bbb{V}[Y^*] = \Bbb{V}[Y] - \frac{[\textrm{cov}(Y,Z)]^2}{\Bbb{V}[Z]} = (1 - \rho_{Y,Z}^2)\Bbb{V}[Y]
$$

The term $\rho_{Y,Z}$ is the correlation coefficient of $Y$ and $Z$

**Control Variates** 

The correlation coefficient is bounded between $-1$ and $1$ with $\rho = 0$ indicating no correlation between $Y$ and $Z$. 

The stronger the correlation between $Y$ and $Z$ (positively or negatively) the more the variance is reduced. 

**Control Variates** 

**Example**: Consider again using MC to estimate 

$$I = \displaystyle\int_0^1 f(x) ~dx = \displaystyle\int_0^1 \frac{1}{1 + x} ~dx$$

This time we'll use as a control variate $g(x) = 1 + x$ which has known expectation $\Bbb{E}[g(X)] = 3/2$

The control variate estimator is then 

$$
\hat{I}^{CV}_N = \frac{1}{N} \sum_{i=1}^N f(X_i) + c\left( \frac{1}{N} \sum_{i=1}^N g(X_i) - \frac{3}{2}\right)
$$



**Control Variates** 

With $N = 1500$ samples we obtain the following 

$$
\begin{array}{|l|r|r|}
\hline
 & \textrm{Estimate} & \textrm{Variance} \\
\hline
\textrm{Vanilla MC} & 0.69513 & 0.02102 \\
\hline
\textrm{Control Variate} & 0.69175 & 0.00055 \\
\hline
\end{array}
$$

This indicates that to get down to a prescribed RMSE, the classic Monte Carlo routine would require nearly 40 times as many samples as the Control Variate routine. 

In [10]:
def g(x):
    return x + 1 